### 0. Importing Necessary Libraries & Dependencies 

In [6]:
##-- Import libraries --##
import torch
import torchvision
from torch import nn
import matplotlib.pyplot as plt
from torchvision import transforms
from helper_modules import data_setup, engine, utils

##-- Setup device agnostic code --##
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"PyTorch Version: {torch.__version__}")
print(f"Device: {device}")

BATCH_SIZE = 64
RANDOM_STATE = 42

PyTorch Version: 2.5.1+cu124
Device: cuda


### 1. Get Data

In [7]:
##-- Setup directory paths --##
from pathlib import Path
image_data_path = Path(r"C:\Users\nextt\Desktop\Hawking\eyeDx-research\aug_train_val")
train_dir = image_data_path / "train"
val_dir = image_data_path / "val"

train_dir, val_dir

(WindowsPath('C:/Users/nextt/Desktop/Hawking/eyeDx-research/aug_train_val/train'),
 WindowsPath('C:/Users/nextt/Desktop/Hawking/eyeDx-research/aug_train_val/val'))

### 2. Create Datasets & DataLoaders

#### 2.1 Creating a transforms for `torchvision.models` (manual creation)

In [8]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.465, 0.406],
                                 std=[0.229, 0.224, 0.225])

manual_tranforms = transforms.Compose([transforms.Resize((600, 600)),
                                       transforms.ToTensor(),
                                       normalize])

#### 2.2 Creating a transforms for `torchvision.models` (auto creation)

- As of `torchvision` v0.13_ there is now support for automatic data transform creation based on the pretrained model weights you're using.

In [9]:
##-- Get a set of pretrained model weights --##
weights_b7 =torchvision.models.EfficientNet_B7_Weights.DEFAULT # 'DEFAULT' = best availabel weight
weights_b7

EfficientNet_B7_Weights.IMAGENET1K_V1

In [10]:
##-- Get the transforms used to create the pretrained weights
auto_transforms = weights_b7.transforms()
auto_transforms

ImageClassification(
    crop_size=[600]
    resize_size=[600]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [11]:
##-- Create DataLoaders using auto_transforms --##
train_dataloader, val_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                              val_dir=val_dir,
                                                                              transform=auto_transforms,
                                                                              batch_size=BATCH_SIZE)
train_dataloader, val_dataloader, class_names


(<torch.utils.data.dataloader.DataLoader at 0x150692cbb90>,
 ['Central Serous Chorioretinopathy [Color Fundus]',
  'Diabetic Retinopathy',
  'Disc Edema',
  'Glaucoma',
  'Healthy',
  'Macular Scar',
  'Myopia',
  'Pterygium',
  'Retinal Detachment',
  'Retinitis Pigmentosa'])

### 3. Getting Pretrained Models

#### 3.1 Setting Up Pretrained Model

In [12]:
import torchvision
weights_b7 = torchvision.models.EfficientNet_B7_Weights.DEFAULT # "DEFAULT" = get the best available weights
model_b7 = torchvision.models.efficientnet_b7(weights=weights_b7).to(device)

In [13]:
print(f"{model_b7.avgpool}\n")
print(f"{model_b7.classifier}")

AdaptiveAvgPool2d(output_size=1)

Sequential(
  (0): Dropout(p=0.5, inplace=True)
  (1): Linear(in_features=2560, out_features=1000, bias=True)
)


#### 3.4 Getting Model Summary Using `torchinfo`

In [14]:
from torchinfo import summary

summary(model=model_b7,
        input_size=(1, 3, 600, 600), # [batch_size, color_channels, height, width],
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 600, 600]     [1, 1000]            --                   True
├─Sequential (features)                                      [1, 3, 600, 600]     [1, 2560, 19, 19]    --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 600, 600]     [1, 64, 300, 300]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 600, 600]     [1, 64, 300, 300]    1,728                True
│    │    └─BatchNorm2d (1)                                  [1, 64, 300, 300]    [1, 64, 300, 300]    128                  True
│    │    └─SiLU (2)                                         [1, 64, 300, 300]    [1, 64, 300, 300]    --                   --
│    └─Sequential (1)                                        [1, 64, 300, 300]    [1, 32, 300,

#### 3.5 Freezing Base Model & Changing Output Layer Based On Our Needs

In [15]:
##-- Freeze all the base layers in EffNetB7 --##
for params in model_b7.features.parameters():
    params.requires_grad = False # won't updated the weights

In [16]:
##-- Update the classifier head of our model to suite our problem --##
from torch import nn

torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)

model_b7.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=2560, # features vector coming in from the forzen layers
              out_features=len(class_names)).to(device))

In [17]:
summary(model=model_b7,
        input_size=(1, 3, 600, 600), # [batch_size, color_channels, height, width],
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 600, 600]     [1, 10]              --                   Partial
├─Sequential (features)                                      [1, 3, 600, 600]     [1, 2560, 19, 19]    --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 600, 600]     [1, 64, 300, 300]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 600, 600]     [1, 64, 300, 300]    (1,728)              False
│    │    └─BatchNorm2d (1)                                  [1, 64, 300, 300]    [1, 64, 300, 300]    (128)                False
│    │    └─SiLU (2)                                         [1, 64, 300, 300]    [1, 64, 300, 300]    --                   --
│    └─Sequential (1)                                        [1, 64, 300, 300]    [1, 3

### 4. Train Model

In [18]:
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)

##-- Define hyperparamters --##
EPOCHS = 20
LEARNING_RATE = 0.001

##-- Define Loss & Optimizer --##
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_b7.parameters(), lr=LEARNING_RATE)

##-- Start timer --##
from timeit import default_timer as timer
start_time = timer()

##-- Setup training and save the results --##
model_b7_results = engine.train(model=model_b7,
                               train_dataloader=train_dataloader,
                               val_dataloader=val_dataloader,
                               optimizer=optimizer,
                               loss_fn=loss_fn,
                               epochs=EPOCHS,
                               device=device)

##-- End timer --##
end_time = timer()

  0%|          | 0/20 [05:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
##- Save the model --##
utils.save_model(model=model_b7,
                 target_dir="./saved_models",
                 model_name=f"efficientnetb7_{LEARNING_RATE}_{EPOCHS}_{BATCH_SIZE}.pth",
                 optimizer=optimizer,
                 epoch=EPOCHS)

print(f"[INFO]: Total training time: {(end_time-start_time)/60:.3f} minutes.")

### 5. Evaluate Model

In [2]:
##-- Plot loss curves --##
epochs = [i for i in range(1, EPOCHS+1)]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, model_b7_results["train_loss"], color="blue", label="train_loss")
plt.plot(epochs, model_b7_results["val_loss"], color="red", label="val_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, model_b7_results["train_acc"], color="blue", label="train_acc")
plt.plot(epochs, model_b7_results["val_acc"], color="green", label="val_acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

NameError: name 'EPOCHS' is not defined

### 6. Make Predictions From Test Set

- To do this we have to make sure that our test/custom data is:
    * Same shape
    * Same device
    * Same datatype...